In [14]:
import re
import os
import unicodedata
import numpy as np
import pandas as pd
from typing import Iterable, Optional, Set
# ===== Ruta B (LDA/TF-IDF): tokens lematizados y sin stopwords =====
import sys, spacy
from spacy.cli import download
download("es_core_news_sm")
nlp = spacy.load("es_core_news_sm", disable=["ner","parser"])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 15.3 MB/s  0:00:00 eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [15]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

def load_jsonl_to_df(path_jsonl: str) -> pd.DataFrame:
    df = pd.read_json(path_jsonl, lines=True)
    if "fecha" in df.columns:
        df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
    return df

def structural_cleaning(df: pd.DataFrame, keep_cols: Optional[Iterable[str]] = None) -> pd.DataFrame:
    df = df.copy()
    if keep_cols is not None:
        keep_cols = [c for c in df.columns if c in keep_cols]
        df = df[keep_cols].copy()
    if "identificador" in df.columns:
        df = df.drop_duplicates(subset=["identificador"])
    if "texto_limpio" in df.columns:
        df = df.drop_duplicates(subset=["texto_limpio"])
    return df.reset_index(drop=True)

def empty_to_nan(df, cols=None, extra_empty_tokens=None):
    df = df.copy()
    if cols is None:
        cols = df.select_dtypes(include=["object", "string"]).columns.tolist()

    # Tokens a tratar como vacíos
    tokens = {"", " ", "null", "NULL", "None", "none", "N/A", "NA", "—", "-", "–"}
    if extra_empty_tokens:
        tokens |= set(extra_empty_tokens)

    # strip() y reemplazo por NaN
    for c in cols:
        df[c] = df[c].astype("string")  # preserva NaNs y permite .str ops
        df[c] = df[c].str.strip()
        df[c] = df[c].replace(list(tokens), np.nan)

    return df


def analyze_missingness_then_drop_small(
    df: pd.DataFrame, threshold_pct: float = 1.0, critical_cols=None, normalize_empty=True, extra_empty_tokens=None):
    if normalize_empty:
        df = empty_to_nan(df, extra_empty_tokens=extra_empty_tokens)

    total_rows = max(len(df), 1)
    missing_pct = (df.isna().sum() / total_rows * 100).round(2).sort_values(ascending=False)

    # 1) Quita filas con nulos en columnas críticas
    if critical_cols:
        df = df.dropna(subset=list(critical_cols))

    # 2) Para columnas con pocos nulos, elimina esas filas nulas
    low_na_cols = missing_pct[missing_pct < threshold_pct].index.tolist()
    for c in low_na_cols:
        df = df[df[c].notna()]

    # Recalcular reporte tras los drops
    total_rows2 = max(len(df), 1)
    missing_pct_after = (df.isna().sum() / total_rows2 * 100).round(2).sort_values(ascending=False)

    return df, missing_pct_after


def _pop_last_df_ref() -> Optional[pd.DataFrame]:
    return globals().pop("_last_df_ref", None)


def fill_missing_logical(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "fecha" in df.columns and "mes" in df.columns:
        mask = df["mes"].isna() & df["fecha"].notna()
        df.loc[mask, "mes"] = pd.to_datetime(df.loc[mask, "fecha"], errors="coerce").dt.strftime("%Y-%m")
    if "texto_limpio" in df.columns:
        df["texto_limpio"] = df["texto_limpio"].fillna("")
    if "tematica" in df.columns:
        df["tematica"] = df["tematica"].fillna("Desconocido")
    return df

def detect_inconsistencies(df: pd.DataFrame) -> pd.DataFrame:
    issues = {}
    if "fecha" in df.columns and "mes" in df.columns:
        f = pd.to_datetime(df["fecha"], errors="coerce")
        m_calc = f.dt.strftime("%Y-%m")
        issues["mes_mismatch"] = (df["mes"].notna()) & (m_calc.notna()) & (df["mes"] != m_calc)
    if "trimestre" in df.columns:
        valid_trim = {"Q1","Q2","Q3","Q4", None, np.nan}
        issues["trimestre_invalid"] = ~df["trimestre"].isin(valid_trim)
    if "texto_limpio" in df.columns:
        issues["texto_muy_corto"] = df["texto_limpio"].str.len().fillna(0) < 20
    inc = pd.DataFrame(issues) if issues else pd.DataFrame(index=df.index)
    if not inc.empty:
        inc["any_issue"] = inc.any(axis=1)
    return inc

RE_URL = re.compile(r'https?://\S+|www\.\S+')
RE_HTML = re.compile(r'<[^>]+>')
RE_EMAIL = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b')
RE_PHONE = re.compile(r'(?:(?:\+?\d{1,3})?[\s.-]?)?(?:\(?\d{2,3}\)?[\s.-]?)?\d{3,4}[\s.-]?\d{3,4}')
RE_NIF = re.compile(r'\b[XYZ]?\d{5,8}[A-Z]\b')

def _clean_text_minimal(s: str, strip_pii: bool = True) -> str:
    if not isinstance(s, str):
        return ""
    x = s
    x = RE_URL.sub(" ", x)
    x = RE_HTML.sub(" ", x)
    if strip_pii:
        x = RE_EMAIL.sub(" ", x)
        x = RE_PHONE.sub(" ", x)
        x = RE_NIF.sub(" ", x)
    x = unicodedata.normalize("NFKC", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

def minimal_text_clean(df: pd.DataFrame, text_col: str = "texto_limpio", strip_pii: bool = True) -> pd.DataFrame:
    df = df.copy()
    if text_col in df.columns:
        df[text_col] = df[text_col].map(lambda s: _clean_text_minimal(s, strip_pii=strip_pii))
    return df

def configure_spacy(extra_stop: Optional[Iterable[str]] = None):
    import spacy
    nlp = spacy.load("es_core_news_sm", disable=["ner","parser"])
    STOP_ES = set(nlp.Defaults.stop_words)
    if extra_stop:
        STOP_ES |= set(extra_stop)
    return nlp, STOP_ES

def _expand_contractions_es(s: str) -> str:
    if not isinstance(s, str):
        return s
    s = re.sub(r"\bdel\b", "de el", s)
    s = re.sub(r"\bal\b", "a el", s)
    return s

def preprocess_for_bow(text: str, nlp, STOP_ES: Set[str]) -> list:
    if not isinstance(text, str) or not text.strip():
        return []
    t = _expand_contractions_es(text.lower())
    doc = nlp(t)
    toks = []
    for w in doc:
        if w.is_punct or w.is_space or w.is_digit:
            continue
        lemma = w.lemma_.strip()
        if len(lemma) < 2:
            continue
        if lemma in STOP_ES:
            continue
        toks.append(lemma)
    return toks

def run_bow_pipeline(df: pd.DataFrame, nlp, STOP_ES: set, text_col: str = "texto_limpio", suffix: str = "") -> pd.DataFrame:
    
    df = df.copy()

    # Procesamiento robusto: evita errores con valores NaN o no string
    df[text_col] = df[text_col].fillna("").astype(str)

    # Tokenización y limpieza
    df[f"tokens_{suffix or text_col}"] = df[text_col].map(
        lambda t: preprocess_for_bow(t, nlp=nlp, STOP_ES=STOP_ES)
    )
    df[f"n_tokens_{suffix or text_col}"] = df[f"tokens_{suffix or text_col}"].map(len)

    return df


def save_outputs(
    df: pd.DataFrame,
    path_parquet: str,
    path_missing_csv: Optional[str] = None,
    path_inconsistencies_csv: Optional[str] = None,
    inconsistencies_df: Optional[pd.DataFrame] = None
):
    os.makedirs(os.path.dirname(path_parquet) or ".", exist_ok=True)
    df.to_parquet(path_parquet, index=False)
    if path_missing_csv:
        os.makedirs(os.path.dirname(path_missing_csv) or ".", exist_ok=True)
        na_pct = (df.isna().sum() / max(len(df),1) * 100).round(2)
        na_pct.to_csv(path_missing_csv, header=["pct_missing"])
    if path_inconsistencies_csv and inconsistencies_df is not None and not inconsistencies_df.empty:
        os.makedirs(os.path.dirname(path_inconsistencies_csv) or ".", exist_ok=True
        )
        inconsistencies_df.to_csv(path_inconsistencies_csv, index=False)


In [16]:
from datasets import load_dataset
# 1) Cargar
dataset = load_dataset("Joz16gg162/boe_2024_dataset", split="full")
df = dataset.to_pandas()
df.head()

,identificador,fecha,diario_numero,seccion_codigo,seccion_nombre,departamento_nombre,epigrafe_nombre,titulo,tematica,texto_limpio,mes,trimestre
0,BOE-A-2024-1,2024-01-01,1,2A,II. Autoridades y personal. - A. Nombramientos...,MINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL,Destinos,"Resolución de 21 de diciembre de 2023, de la S...",Otras,Por Resolución de la Subsecretaría de este Dep...,2024-01,Q1
1,BOE-A-2024-2,2024-01-01,1,2A,II. Autoridades y personal. - A. Nombramientos...,UNIVERSIDADES,Nombramientos,"Resolución de 21 de diciembre de 2023, de la U...",Economía/Empresa,Vistas las propuestas elevadas por las comisio...,2024-01,Q1
2,BOE-A-2024-3,2024-01-01,1,2A,II. Autoridades y personal. - A. Nombramientos...,UNIVERSIDADES,Nombramientos,"Resolución de 22 de diciembre de 2023, conjunt...",Sanidad,Vista la propuesta elevada el 18 de diciembre ...,2024-01,Q1
3,BOE-A-2024-4,2024-01-01,1,2A,II. Autoridades y personal. - A. Nombramientos...,UNIVERSIDADES,Nombramientos,"Resolución de 22 de diciembre de 2023, de la U...",Economía/Empresa,De conformidad con la propuesta elevada por la...,2024-01,Q1
4,BOE-A-2024-5,2024-01-01,1,2A,II. Autoridades y personal. - A. Nombramientos...,UNIVERSIDADES,Nombramientos,"Resolución de 22 de diciembre de 2023, de la U...",Educación/Universidad,De conformidad con la propuesta elevada por la...,2024-01,Q1


In [17]:
# 2) Limpieza estructural
KEEP = {
    "identificador","fecha","mes","seccion_nombre",
    "departamento_nombre","epigrafe_nombre","titulo","texto_limpio"
}
df = structural_cleaning(df, keep_cols=KEEP)

In [18]:
df.shape

(75382, 8)

In [19]:
# 3) Analizar faltantes y eliminar filas con ausencias pequeñas
df, missing_report = analyze_missingness_then_drop_small(
    df,
    threshold_pct=1.0,
    critical_cols=["identificador", "texto_limpio", "epigrafe_nombre"],
    normalize_empty=True
)


In [20]:
df.shape

(27622, 8)

In [21]:
# 4) Rellenar lo que sí tiene sentido (mes desde fecha, tematica vacía -> "Desconocido", etc.)
df = fill_missing_logical(df)

In [22]:
# 5) Inconsistencias (mes vs fecha, trimestre válido, texto corto)
inc = detect_inconsistencies(df)

In [23]:
# 6) Limpieza mínima de texto (URLs/HTML/PII + espacios)
df = minimal_text_clean(df, text_col="texto_limpio", strip_pii=True)

In [26]:
# ===== Ruta A (Transformers/LLMs): mantiene texto natural =====
save_outputs(
    df,
    path_parquet="data/boe_2024_clean.parquet",
    path_missing_csv="data/missing_report.csv",
    path_inconsistencies_csv="data/inconsistencies_report.csv",
    inconsistencies_df=inc
)


In [27]:
'''nlp, STOP_ES = configure_spacy(extra_stop={
    "artículo","disposición","resuelve","boe","anexo","rector",
    "subsecretaría","tribunal","resolución","convocatoria","administrativo"
})
# Tokenizar solo texto_limpio
df_texto = run_bow_pipeline(df, nlp=nlp, STOP_ES=STOP_ES, text_col="texto_limpio", suffix="texto")

# Tokenizar solo titulo
df_titulo = run_bow_pipeline(df, nlp=nlp, STOP_ES=STOP_ES, text_col="titulo", suffix="titulo")

# Unir resultados
df_bow = df[["identificador", "titulo", "texto_limpio"]] \
    .merge(df_texto[["identificador", "tokens_texto"]], on="identificador", how="left") \
    .merge(df_titulo[["identificador", "tokens_titulo"]], on="identificador", how="left")

# Guardar
save_outputs(df_bow, path_parquet="data/processed/boe_2024_tokens_titulo_texto.parquet")
'''



'nlp, STOP_ES = configure_spacy(extra_stop={\n    "artículo","disposición","resuelve","boe","anexo","rector",\n    "subsecretaría","tribunal","resolución","convocatoria","administrativo"\n})\n# Tokenizar solo texto_limpio\ndf_texto = run_bow_pipeline(df, nlp=nlp, STOP_ES=STOP_ES, text_col="texto_limpio", suffix="texto")\n\n# Tokenizar solo titulo\ndf_titulo = run_bow_pipeline(df, nlp=nlp, STOP_ES=STOP_ES, text_col="titulo", suffix="titulo")\n\n# Unir resultados\ndf_bow = df[["identificador", "titulo", "texto_limpio"]]     .merge(df_texto[["identificador", "tokens_texto"]], on="identificador", how="left")     .merge(df_titulo[["identificador", "tokens_titulo"]], on="identificador", how="left")\n\n# Guardar\nsave_outputs(df_bow, path_parquet="data/processed/boe_2024_tokens_titulo_texto.parquet")\n'